<a href="https://colab.research.google.com/github/LeszekRutkowski/dw_matrix_road_sign/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install hyperopt

In [18]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure 

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [3]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')


X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train,
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose', 1),
      validation_data=params_fit.get('validation_data', (X_train, y_train)),
      callbacks= [tensorboard_callback]

  )
  return model

  


In [0]:
def predict(model_trained, X_test, y_test, scoring=accuracy_score):

  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(0.3),


    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(1024, activation='relu'),
    Dropout(0.3),


    Dense(num_classes, activation='softmax'),
  ])





In [13]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)
predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 4s 15ms/step - loss: 2.6786 - accuracy: 0.2788 - val_loss: 1.1860 - val_accuracy: 0.6205
Epoch 2/5
272/272 [==============================] - 4s 14ms/step - loss: 0.9959 - accuracy: 0.6834 - val_loss: 0.3676 - val_accuracy: 0.8721
Epoch 3/5
272/272 [==============================] - 4s 14ms/step - loss: 0.5110 - accuracy: 0.8371 - val_loss: 0.1720 - val_accuracy: 0.9460
Epoch 4/5
272/272 [==============================] - 4s 14ms/step - loss: 0.3330 - accuracy: 0.8964 - val_loss: 0.0924 - val_accuracy: 0.9732
Epoch 5/5
272/272 [==============================] - 4s 14ms/step - loss: 0.2370 - accuracy: 0.9273 - val_loss: 0.0687 - val_accuracy: 0.9803


0.9517006802721089

In [15]:
model_trained.evaluate(X_test, y_test)[1]

138/138 [==============================] - 0s 3ms/step - loss: 0.2055 - accuracy: 0.9517


0.9517006874084473

In [0]:
def get_model(params):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),


    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_two']),

    Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_three']),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_one']),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_two']),


    Dense(num_classes, activation='softmax'),
  ])





In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  


  model.fit(
      X_train,
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=5,
      verbose=0
      
  )

  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model':model}

In [35]:
space = {
    'batch_size' : hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one' : hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two' : hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three' : hp.uniform('dropout_cnn_block_three', 0.3, 0.5),

    'dropout_dense_block_one' : hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two' : hp.uniform('dropout_dense_block_two', 0.3, 0.7),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 180.0, 'dropout_cnn_block_one': 0.4859147982867026, 'dropout_cnn_block_three': 0.4394940269618302, 'dropout_cnn_block_two': 0.4240508214576322, 'dropout_dense_block_one': 0.39322580272426766, 'dropout_dense_block_two': 0.31792299255474826}
accuracy=0.8727891445159912
{'batch_size': 180.0, 'dropout_cnn_block_one': 0.38772514911402106, 'dropout_cnn_block_three': 0.4900479167645517, 'dropout_cnn_block_two': 0.38071522551250464, 'dropout_dense_block_one': 0.6388629427333297, 'dropout_dense_block_two': 0.5119074398148835}
accuracy=0.8619047403335571
{'batch_size': 120.0, 'dropout_cnn_block_one': 0.4819687593017937, 'dropout_cnn_block_three': 0.3237997883270203, 'dropout_cnn_block_two': 0.47643557135595427, 'dropout_dense_block_one': 0.6002203299617768, 'dropout_dense_block_two': 0.32205551659074816}
accuracy=0.9510204195976257
{'batch_size': 150.0, 'dropout_cnn_block_one': 0.47866735685439804, 'dropout_cnn_block_three': 0.4311686784078968, 'dropout_cnn_block_two': 0.385624625